In [1]:
from together import Together

TOGETHER_API_KEY = "5a532872525382e32ebc396c6cc682d3b8d8d5ea428ef9468404286bb1417f2c"

class GooperBackendModel:
    def __init__(self):
        self.client = Together(api_key=TOGETHER_API_KEY)
        self.get_post_description_system_prompt = open("../prompts/describe_image.txt", "r").read()
        print(self.get_post_description_system_prompt + '\n')

    def get_post_description(self, imageUrl):
        
        systemPrompt = self.get_post_description_system_prompt
        
        response = self.client.chat.completions.create(
            model="meta-llama/Llama-Vision-Free",
            messages=[
                {   
                    "role": "system",
                    "content": [
                        {"type": "text", "text": systemPrompt},
                    ],
                    "role": "user",
                    "content": [
                        {"type": "text", "text": systemPrompt},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": imageUrl,
                            },
                        },
                    ],
                }
            ],
        )
        
        return response.choices[0].message.content

# model = GooperBackendModel()
# print(model.get_post_description("https://scontent-man2-1.cdninstagram.com/v/t51.2885-15/53687533_1525053140964204_8915261529964025056_n.jpg?stp=dst-jpg_e35_tt6&_nc_ht=scontent-man2-1.cdninstagram.com&_nc_cat=100&_nc_ohc=H_PaUbTfdZYQ7kNvgESuPtM&_nc_gid=431ddddf6602442aad49ec798f4e18f7&edm=AOQ1c0wBAAAA&ccb=7-5&ig_cache_key=MjAwNzM2NjM3NTEyODIwMDg1MA%3D%3D.3-ccb7-5&oh=00_AYBXtaVOQ6LaVNSgZ4f7dZuomtzSRE4q2wVYcPLQVfbUWQ&oe=676A9240&_nc_sid=8b3546"))

In [2]:
import pandas as pd
import json

In [3]:
data = json.load(open('241221_apify.json'))

From apify .json dataset create a custom polished dataset

In [4]:
df = []

for i in range(len(data)):
    try:
        entry = {
            'url': data[i]['inputUrl'],
            'username': data[i]['username'],
            'biography': data[i]['biography'],
            'followersCount': data[i]['followersCount'],
            'followsCount': data[i]['followsCount'],
            'businessCategoryName': data[i]['businessCategoryName'],
            'latestPosts': []
        }

        # keep only those with [2k, 10k] followers
        if entry['followersCount'] < 2000 or entry['followersCount'] > 10000:
            continue

        # retrieve last 5 post images and captions
        for j in range(len(data[i]['latestPosts'])):
            
            if data[i]['latestPosts'][j]['type'] == 'video':
                continue

            if len(entry['latestPosts']) >= 5:
                continue

            entry['latestPosts'].append({'displayUrl': data[i]['latestPosts'][j]['displayUrl'],
                                        'caption': data[i]['latestPosts'][j]['caption'],
                                        })
            
        if len(entry['latestPosts']) < 5:
            continue

        df.append(entry)

    except:
        print(f'Failed at {i}')

print(f'Successfully saved {len(df)} entries')    

Failed at 168
Failed at 175
Failed at 667
Successfully saved 90 entries


Get description of each image

In [ ]:
import time

model = GooperBackendModel()

for i in range(len(df)):
    
    print('-'*50)
    print(f'Processing profile {i+1:03}/{len(df)+1}')
    start = time.time()

    for j in range(len(df[i]['latestPosts'])):
        try:
            df[i]['latestPosts'][j]['description'] = model.get_post_description(df[i]['latestPosts'][j]['displayUrl'])
            
            print(f'\t{j}. ', df[i]['latestPosts'][j]['description'])
        except Exception as e:
            print(f'Failed at {i} {j}: {e}')

    end = time.time()
    
    print(f'Completed in {end-start:.2f}s')

with open(f'241221_apify_desc_{int(time.time())}.json', 'w') as f:
    json.dump(df, f)

You are a social media analyst and your task is to summarize the content and context of a given image, which represents an Instagram post.
Describe the main objects, people, or scenes in the image, including their appearance, actions, and expressions.
Identify the mood, aesthetic, or theme of the image and any notable unique features.  
Make the summary concise yet detailed enough to help understand the overall content and intent of the post.

Guidelines:
- Do not use bullet points, titles or lists.
- Write a single sentence starting with "This image..."
- Use not more than 30 words

--------------------------------------------------
Processing profile 001/91
	0.  This image depicts a woman in a business suit running through an urban alleyway, capturing a moment of action and movement.
	1.  The image shows a poster for an album by the band The Zen Circus, featuring a gold record and a photo of three people in a natural setting, with text in Italian.
	2.  This image captures a musician 